In [10]:
import plotly.express as px
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.layers import Dense
from keras.layers import Flatten, Dropout, Normalization
from keras.models import Model
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.models import load_model
from utils.functions import *
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

In [11]:
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print("NUM GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

NUM GPUs Available:  1


In [12]:
from tensorflow.keras.applications import ResNet50
from keras.applications.resnet import preprocess_input

model = ResNet50(include_top=False, input_shape=(112, 112,3))
model_name = "ResNet50"

In [13]:
train_path = "data/train"
test_path = "data/test"
labels = {"akiec":0, "bcc":1, "bkl":2, "df":3, "mel":4, "nv":5, "vasc":6}
classes = list(labels.keys())
size = (112,112)

In [14]:
model_generator = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range = 40,
                                        width_shift_range = 0.2, height_shift_range = 0.2, rescale = 1./255,
                                        shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True,
                                        validation_split=.2)

train_batches = model_generator.flow_from_directory(train_path,target_size=size,classes=classes,
                                                batch_size=40, shuffle=True, subset="training")
validation_batches = model_generator.flow_from_directory(train_path,size,classes=classes,
                                                batch_size=20, shuffle=True, subset="validation")
test_batches = ImageDataGenerator(preprocessing_function=preprocess_input)\
    .flow_from_directory(test_path,size,classes=classes, batch_size=10, shuffle=False)

Found 3763 images belonging to 7 classes.
Found 937 images belonging to 7 classes.
Found 2003 images belonging to 7 classes.


In [15]:
flat1 = Flatten()(model.layers[-1].output)
norm = Normalization()(flat1)
class1 = Dense(128, activation="relu")(norm)
class2 = Dropout(rate=.3)(class1)
class3 = Dense(64, activation="relu")(class2)
class4 = Dropout(rate=.3)(class3)
class5 = Dense(32, activation="relu")(class4)
class6 = Dropout(rate=.3)(class5)
output = Dense(7, activation="softmax")(class6)
model = Model(inputs=model.inputs, outputs=output)

In [16]:
learning_rate = 0.06
optimizer = Adam(learning_rate)
loss = "categorical_crossentropy"
metrics = "accuracy"

model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=metrics)

In [17]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
mc = ModelCheckpoint('/deployment/models/'+model_name+'_training_{accuracy:.4f}_{val_accuracy:.4f}.h5',
                     monitor='val_accuracy', mode='max', verbose=1,
                     save_best_only=True)
lrl = ReduceLROnPlateau(patience=10,verbose=1,monitor="loss")

In [18]:
history = model.fit(train_batches, validation_data= validation_batches,
          epochs=100, shuffle=True, verbose=2, callbacks=[es,mc,lrl])

y_pred_proba = model.predict(train_batches)
y_pred = np.argmax(y_pred_proba)

Epoch 1/100
95/95 - 31s - loss: 54.5433 - accuracy: 0.4135 - val_loss: 12575.3438 - val_accuracy: 0.2273

Epoch 00001: val_accuracy improved from -inf to 0.22732, saving model to /deployment/models\ResNet50_training_0.4135_0.2273.h5
Epoch 2/100
95/95 - 26s - loss: 13.8232 - accuracy: 0.4279 - val_loss: 1.5316 - val_accuracy: 0.4376

Epoch 00002: val_accuracy improved from 0.22732 to 0.43757, saving model to /deployment/models\ResNet50_training_0.4279_0.4376.h5
Epoch 3/100
95/95 - 26s - loss: 2.3323 - accuracy: 0.4337 - val_loss: 1.5322 - val_accuracy: 0.4376

Epoch 00003: val_accuracy did not improve from 0.43757
Epoch 4/100
95/95 - 26s - loss: 2.0837 - accuracy: 0.4366 - val_loss: 1.5292 - val_accuracy: 0.4376

Epoch 00004: val_accuracy did not improve from 0.43757
Epoch 5/100
95/95 - 26s - loss: 6.2680 - accuracy: 0.4353 - val_loss: 1.5309 - val_accuracy: 0.4376

Epoch 00005: val_accuracy did not improve from 0.43757
Epoch 6/100
95/95 - 26s - loss: 1.9251 - accuracy: 0.4366 - val_los

In [21]:
best_model = keep_best_saved_h5("/deployment/models/",model_name,0.2)
model = load_model(f"deployment/models/{best_model}")

Currently in directory:C:\Users\Hoofdgebruiker\Desktop\GNT-Arai-31\challenge-mole
File coming out of the function: ResNet50_training_0.6616_0.6704.h5


In [23]:
print(y_pred_proba)

[[0.05603681 0.08787313 0.18501778 ... 0.18938473 0.4375416  0.02431787]
 [0.05603681 0.08787313 0.18501778 ... 0.18938473 0.4375416  0.02431787]
 [0.05603681 0.08787313 0.18501778 ... 0.18938473 0.4375416  0.02431787]
 ...
 [0.05603681 0.08787313 0.18501778 ... 0.18938473 0.4375416  0.02431787]
 [0.05603681 0.08787313 0.18501778 ... 0.18938473 0.4375416  0.02431787]
 [0.05603681 0.08787313 0.18501778 ... 0.18938473 0.4375416  0.02431787]]
